## Summing all the prime numbers below a given number

In [ ]:
import sys
import os
import time
from numba import njit, prange, set_num_threads

# 1. Captura del límite 'number' (parámetro $1 del script de Bash)
# Si no hay argumento, usamos el valor por defecto de la práctica
number = int(sys.argv[1]) if len(sys.argv) > 1 else 2_500_000

# 2. Captura de CPUs (Variable de entorno de SLURM)
n_cpus = int(os.environ.get('SLURM_CPUS_PER_TASK', '1'))

# Configuramos Numba para usar exactamente los cores asignados
set_num_threads(n_cpus)

print(f"--- INICIANDO EXPERIMENTO ---")
print(f"Límite (number): {number}")
print(f"Núcleos asignados: {n_cpus}")
print(f"-----------------------------")

In [1]:
#código original 
import time

# Simple code

def if_prime(x):
    if x <= 1:
        return 0
    elif x <= 3:
        return x
    elif x % 2 == 0 or x % 3 == 0:
        return 0
    i = 5
    while i**2 <= x:
        if x % i == 0 or x % (i + 2) == 0:
            return 0
        i += 6
    return x

def sum_primes(x):
    result = 0
    for i in range(x):
        result += if_prime(i)
    return result

suma = 0
N = 3 # number of loops

start = time.time()
for i in range(N):
    suma = sum(map(if_prime, list(range(number))))
stop = time.time()
tiempo = (stop - start) / N

print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)

tiempo = %timeit -r 2 -o -q sum_primes(number)
suma = sum_primes(number)
print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)


The prime sum below  2500000 is  219697708195  and the time taken is 4.72970453898112
The prime sum below  2500000 is  219697708195  and the time taken is 4.58 s ± 1.25 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


In [2]:
# 3.2: apartado a) - Optimización Numba Secuencial
@njit
def if_prime(x):
    if x <= 1: return 0
    elif x <= 3: return x
    elif x % 2 == 0 or x % 3 == 0: return 0
    i = 5
    while i**2 <= x:
        if x % i == 0 or x % (i + 2) == 0:
            return 0
        i += 6
    return x

@njit
def sum_primes_seq(x):
    result = 0
    for i in range(x):
        result += if_prime(i)
    return result

# Medición
start = time.time()
suma = sum_primes_seq(number)
tiempo = time.time() - start

print(f"A) Numba Secuencial:")
print(f"Suma: {suma} | Tiempo: {tiempo:.4f} s")

The prime sum below  2500000 is  219697708195  and the time taken is 0.6665724118550619
The prime sum below  2500000 is  219697708195  and the time taken is 183 ms ± 463 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)


In [3]:
# 3.2: apartado b) - Multiprocessing con Pool
from multiprocessing import Pool

# Función wrapper para el worker
def check_prime_worker(x):
    return if_prime(x)

def run_parallel_pool(val, cores):
    # Calculamos un chunksize dinámico para optimizar la comunicación
    # Dividir la carga total entre los cores con un factor de balanceo
    dynamic_chunk = max(1, val // (cores * 4))
    
    with Pool(processes=cores) as pool:
        # Map distribuye la carga según el chunksize 
        partial_results = pool.map(check_prime_worker, range(val), chunksize=dynamic_chunk)
    return sum(partial_results)

if __name__ == '__main__':
    start = time.time()
    suma_mp = run_parallel_pool(number, n_cpus)
    tiempo_mp = time.time() - start
    
    print(f"B) Multiprocessing Pool ({n_cpus} procesos):")
    print(f"Suma: {suma_mp} | Tiempo: {tiempo_mp:.4f} s")

Con multiprocessing Pool (4 procesos):
The prime sum below 2500000 is 219697708195 and the time taken is 0.3059 seconds


In [1]:
# 3.2: apartado c) - Numba con prange y Fastmath
@njit(parallel=True, fastmath=True)
def sum_primes_numba_par(x):
    result = 0
    # prange distribuye el bucle entre los hilos configurados 
    for i in prange(x):
        result += if_prime(i)
    return result

# Warm-up: Compilamos antes de medir para una precisión total
_ = sum_primes_numba_par(1000)

# Medición real
start = time.time()
suma_nb = sum_primes_numba_par(number)
tiempo_nb = time.time() - start

print(f"C) Numba con prange (Parallel=True):")
print(f"Suma: {suma_nb} | Tiempo: {tiempo_nb:.4f} s")

# Medición extra para el reporte
print("\nEjecutando timeit para estadísticas finales...")
%timeit -r 2 -n 5 sum_primes_numba_par(number)

Con Numba prange (4 threads):
The prime sum below 2500000 is 219697708195 and the time taken is 0.0337 seconds
Tiempo con timeit: 33.6 ms ± 3.13 μs per loop (mean ± std. dev. of 2 runs, 10 loops each)


-------------------------------------------------------
Ejecución de Pruebas de Rendimiento - Suma de Primos
Usuario: alumno01 | ID Trabajo: 13543
-------------------------------------------------------

=======================================================
 ANALIZANDO TAMAÑO: 1000000
=======================================================
[INFO] Lanzando con 1 núcleos...
--- INICIANDO EXPERIMENTO ---
Límite (number): 1000000
Núcleos asignados: 1
-----------------------------
The prime sum below  1000000 is  37550402023  and the time taken is 2.2844995657602944
The prime sum below  1000000 is  37550402023  and the time taken is 2.23 s ± 333 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)
A) Numba Secuencial:
Suma: 37550402023 | Tiempo: 1.1048 s
B) Multiprocessing Pool (1 procesos):
Suma: 37550402023 | Tiempo: 0.4586 s
C) Numba con prange (Parallel=True):
Suma: 37550402023 | Tiempo: 0.1147 s

Ejecutando timeit para estadísticas finales...
115 ms ± 3.77 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)
-------------------------------------------------------
[INFO] Lanzando con 2 núcleos...
--- INICIANDO EXPERIMENTO ---
Límite (number): 1000000
Núcleos asignados: 2
-----------------------------
The prime sum below  1000000 is  37550402023  and the time taken is 2.2917868296305337
The prime sum below  1000000 is  37550402023  and the time taken is 2.25 s ± 312 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)
A) Numba Secuencial:
Suma: 37550402023 | Tiempo: 1.2368 s
B) Multiprocessing Pool (2 procesos):
Suma: 37550402023 | Tiempo: 0.2570 s
C) Numba con prange (Parallel=True):
Suma: 37550402023 | Tiempo: 0.0706 s

Ejecutando timeit para estadísticas finales...
70.5 ms ± 7.08 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)
-------------------------------------------------------
[INFO] Lanzando con 4 núcleos...
--- INICIANDO EXPERIMENTO ---
Límite (number): 1000000
Núcleos asignados: 4
-----------------------------
The prime sum below  1000000 is  37550402023  and the time taken is 2.3031179904937744
The prime sum below  1000000 is  37550402023  and the time taken is 2.24 s ± 10.1 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
A) Numba Secuencial:
Suma: 37550402023 | Tiempo: 0.8853 s
B) Multiprocessing Pool (4 procesos):
Suma: 37550402023 | Tiempo: 0.1640 s
C) Numba con prange (Parallel=True):
Suma: 37550402023 | Tiempo: 0.0377 s

Ejecutando timeit para estadísticas finales...
37.6 ms ± 2.31 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)
-------------------------------------------------------
[INFO] Lanzando con 8 núcleos...
--- INICIANDO EXPERIMENTO ---
Límite (number): 1000000
Núcleos asignados: 8
-----------------------------
The prime sum below  1000000 is  37550402023  and the time taken is 2.323868672053019
The prime sum below  1000000 is  37550402023  and the time taken is 2.31 s ± 2 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
A) Numba Secuencial:
Suma: 37550402023 | Tiempo: 0.8632 s
B) Multiprocessing Pool (8 procesos):
Suma: 37550402023 | Tiempo: 0.1376 s
C) Numba con prange (Parallel=True):
Suma: 37550402023 | Tiempo: 0.0195 s

Ejecutando timeit para estadísticas finales...
19.3 ms ± 7.92 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)
-------------------------------------------------------

=======================================================
 ANALIZANDO TAMAÑO: 10000000
=======================================================
[INFO] Lanzando con 1 núcleos...
--- INICIANDO EXPERIMENTO ---
Límite (number): 10000000
Núcleos asignados: 1
-----------------------------
The prime sum below  10000000 is  3203324994356  and the time taken is 60.9279355208079
The prime sum below  10000000 is  3203324994356  and the time taken is 59.7 s ± 30.1 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
A) Numba Secuencial:
Suma: 3203324994356 | Tiempo: 4.0252 s
B) Multiprocessing Pool (1 procesos):
Suma: 3203324994356 | Tiempo: 5.9763 s
C) Numba con prange (Parallel=True):
Suma: 3203324994356 | Tiempo: 2.8716 s

Ejecutando timeit para estadísticas finales...
2.87 s ± 23.6 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)
-------------------------------------------------------
[INFO] Lanzando con 2 núcleos...
--- INICIANDO EXPERIMENTO ---
Límite (number): 10000000
Núcleos asignados: 2
-----------------------------
The prime sum below  10000000 is  3203324994356  and the time taken is 60.567136685053505
The prime sum below  10000000 is  3203324994356  and the time taken is 59.2 s ± 338 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
A) Numba Secuencial:
Suma: 3203324994356 | Tiempo: 4.0526 s
B) Multiprocessing Pool (2 procesos):
Suma: 3203324994356 | Tiempo: 3.2096 s
C) Numba con prange (Parallel=True):
Suma: 3203324994356 | Tiempo: 1.7910 s

Ejecutando timeit para estadísticas finales...
1.79 s ± 137 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)
-------------------------------------------------------
[INFO] Lanzando con 4 núcleos...
--- INICIANDO EXPERIMENTO ---
Límite (number): 10000000
Núcleos asignados: 4
-----------------------------
The prime sum below  10000000 is  3203324994356  and the time taken is 60.35822399457296
The prime sum below  10000000 is  3203324994356  and the time taken is 1min ± 126 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
A) Numba Secuencial:
Suma: 3203324994356 | Tiempo: 4.1130 s
B) Multiprocessing Pool (4 procesos):
Suma: 3203324994356 | Tiempo: 1.8038 s
C) Numba con prange (Parallel=True):
Suma: 3203324994356 | Tiempo: 0.9572 s

Ejecutando timeit para estadísticas finales...
957 ms ± 112 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)
-------------------------------------------------------
[INFO] Lanzando con 8 núcleos...
--- INICIANDO EXPERIMENTO ---
Límite (number): 10000000
Núcleos asignados: 8
-----------------------------
The prime sum below  10000000 is  3203324994356  and the time taken is 61.44812766710917
The prime sum below  10000000 is  3203324994356  and the time taken is 58.6 s ± 14 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
A) Numba Secuencial:
Suma: 3203324994356 | Tiempo: 4.0845 s
B) Multiprocessing Pool (8 procesos):
Suma: 3203324994356 | Tiempo: 1.1321 s
C) Numba con prange (Parallel=True):
Suma: 3203324994356 | Tiempo: 0.4922 s

Ejecutando timeit para estadísticas finales...
492 ms ± 75.5 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)
-------------------------------------------------------
Experimentos finalizados correctamente.


apartado e)

Análisis de Resultados: Suma de Primos (Rendimiento y Escalabilidad)

Tras ejecutar los experimentos en el clúster (Mendel) para límites de 106 y 107 con diferentes configuraciones de núcleos (1,2,4,8), se observan las siguientes conclusiones técnicas sobre los cuatro procedimientos utilizados:

1. Resumen de Tiempos (Caso 10.000.000)
Método	1 Núcleo	2 Núcleos	4 Núcleos	8 Núcleos
Código Original (Python)	≈60.0 s	≈60.0 s	≈60.0 s	≈60.0 s
Numba Secuencial	4.02 s	4.05 s	4.11 s	4.08 s
Multiprocessing Pool	5.97 s	3.20 s	1.80 s	1.13 s
Numba con prange (Par.)	2.87 s	1.79 s	0.95 s	0.49 s

